In [1]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Load data
file_path = Path("Resources/final_sample.csv")
df = pd.read_csv(file_path)
df.head()

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,applicant_ethnicity,co_applicant_ethnicity,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region,action_taken_summary
0,NCUA,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,123.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,2310.0,20.35,65300.0,74.28,941.0,3848.0,MW,0
1,HUD,Conventional,One to Four-Family,Home Purchase,Owner Occupied,387.0,Not Applicable,Application Approved but not Accepted,Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,5192.0,96.51,64300.0,53.53,397.0,1056.0,W,0
2,HUD,Conventional,One to Four-Family,Refinancing,Owner Occupied,376.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,12130.0,66.65,63200.0,112.07,921.0,2099.0,W,0
3,HUD,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,272.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,6335.0,27.73,91500.0,136.37,1406.0,1541.0,MA,0
4,NCUA,Conventional,One to Four-Family,Home Improvement,Owner Occupied,40.0,Not Applicable,Application Approved but not Accepted,Info not Provided by Applicant,No Co-Applicant,...,Not a HOEPA Loan,Secured by a Subordinate Loan,5625.0,5.33,79600.0,153.58,1693.0,1679.0,MW,0


Run Neural Network

In [3]:
# Generate our categorical variable lists
action_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[action_cat].nunique()

agency_code                6
loan_type                  4
property_type              2
loan_purpose               3
owner_occupancy            3
preapproval                3
action_taken               6
applicant_ethnicity        4
co_applicant_ethnicity     5
applicant_race_1           7
co_applicant_race_1        8
applicant_sex              4
co_applicant_sex           5
purchaser_type            10
hoepa_status               2
lien_status                4
Region                     7
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[action_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(action_cat)
encode_df.head()

,agency_code_CFPB,agency_code_FDIC,agency_code_FRS,agency_code_HUD,agency_code_NCUA,agency_code_OCC,loan_type_Conventional,loan_type_FHA-Insured,loan_type_FSA/RHS,loan_type_VA-Guaranteed,...,lien_status_Not Secured by Lien,lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(action_cat,1)
df.head()

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,loan_amount_000s,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,action_taken_summary,agency_code_CFPB,...,lien_status_Not Secured by Lien,lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,123.0,187.0,2310.0,20.35,65300.0,74.28,941.0,3848.0,0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,387.0,148.0,5192.0,96.51,64300.0,53.53,397.0,1056.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,376.0,208.0,12130.0,66.65,63200.0,112.07,921.0,2099.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,272.0,111.0,6335.0,27.73,91500.0,136.37,1406.0,1541.0,0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.0,120.0,5625.0,5.33,79600.0,153.58,1693.0,1679.0,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# check values
df.count()

loan_amount_000s            100725
applicant_income_000s       100725
population                  100725
minority_population         100725
hud_median_family_income    100725
                             ...  
Region_NE                   100725
Region_NW                   100725
Region_SE                   100725
Region_SW                   100725
Region_W                    100725
Length: 92, dtype: int64

loan_amount_000s            100725
applicant_income_000s       100725
population                  100725
minority_population         100725
hud_median_family_income    100725
                             ...  
Region_NE                   100725
Region_NW                   100725
Region_SE                   100725
Region_SW                   100725
Region_W                    100725
Length: 92, dtype: int64

In [7]:
# Split our preprocessed data into our features and target arrays
y = df["action_taken_summary"].values
X = df.drop(["action_taken_summary"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
import keras

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 92
hidden_nodes_layer2 = 61
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 92)                8464      
                                                                 
 dense_1 (Dense)             (None, 61)                5673      
                                                                 
 dense_2 (Dense)             (None, 40)                2480      
                                                                 
 dense_3 (Dense)             (None, 1)                 41        
                                                                 
Total params: 16,658
Trainable params: 16,658
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
2361/2361 [==============================] - 8s 3ms/step - loss: 24.2699 - accuracy: 0.6680
Epoch 2/100
2361/2361 [==============================] - 6s 3ms/step - loss: 1.5754 - accuracy: 0.7059
Epoch 3/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.6408 - accuracy: 0.7550
Epoch 4/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.6984 - accuracy: 0.7721
Epoch 5/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.5284 - accuracy: 0.7789
Epoch 6/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.5283 - accuracy: 0.7789
Epoch 7/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.5284 - accuracy: 0.7789
Epoch 8/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.5283 - accuracy: 0.7789
Epoch 9/100
2361/2361 [==============================] - 6s 3ms/step - loss: 0.5284 - accuracy: 0.7789
Epoch 10/100
2361/2361 [==============================] - 6s 3ms/step - 

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

787/787 - 2s - loss: 0.5477 - accuracy: 0.7821 - 2s/epoch - 3ms/step
Loss: 0.5476956963539124, Accuracy: 0.7820665836334229
